### xarray wrapper for pyqg
https://github.com/pyqg/pyqg/issues/145

In [196]:
import numpy as np
import xarray as xr
import pyqg
from pyqg import diagnostic_tools
import matplotlib.pyplot as plt

In [85]:
year = 24*60*60*360.
m = pyqg.QGModel(tmax=10*year, twrite=10000, tavestart=5*year)
m.run()

for v in dir(m):
    print(f"{v}:{type(getattr(m,v))}") 

INFO:  Logger initialized
INFO: Step: 10000, Time: 7.20e+07, KE: 5.03e-04, CFL: 0.082
INFO: Step: 20000, Time: 1.44e+08, KE: 5.05e-04, CFL: 0.088
INFO: Step: 30000, Time: 2.16e+08, KE: 4.78e-04, CFL: 0.085
INFO: Step: 40000, Time: 2.88e+08, KE: 4.36e-04, CFL: 0.077


F1:<class 'float'>
F2:<class 'float'>
H:<class 'numpy.float64'>
Hi:<class 'numpy.ndarray'>
Jptpc:<class 'numpy.ndarray'>
Jpxi:<class 'numpy.ndarray'>
L:<class 'float'>
M:<class 'int'>
Qy:<class 'numpy.ndarray'>
Qy1:<class 'float'>
Qy2:<class 'float'>
U:<class 'float'>
U1:<class 'float'>
U2:<class 'float'>
Ubg:<class 'numpy.ndarray'>
W:<class 'float'>
__class__:<class 'type'>
__delattr__:<class 'method-wrapper'>
__dict__:<class 'dict'>
__dir__:<class 'builtin_function_or_method'>
__doc__:<class 'str'>
__eq__:<class 'method-wrapper'>
__format__:<class 'builtin_function_or_method'>
__ge__:<class 'method-wrapper'>
__getattribute__:<class 'method-wrapper'>
__gt__:<class 'method-wrapper'>
__hash__:<class 'method-wrapper'>
__init__:<class 'method'>
__init_subclass__:<class 'builtin_function_or_method'>
__le__:<class 'method-wrapper'>
__lt__:<class 'method-wrapper'>
__module__:<class 'str'>
__ne__:<class 'method-wrapper'>
__new__:<class 'builtin_function_or_method'>
__pyx_vtable__:<class 'PyCa

In [86]:
# help(pyqg.Model)
# help(pyqg.QGModel)
# help(pyqg.LayeredModel)
# help(pyqg.BTModel)
# help(pyqg.SQGModel)
# help(pyqg.LagrangianParticleArray2D)
# help(pyqg.GriddedLagrangianParticleArray2D)
# help(diagnostic_tools.calc_ispec)
# help(diagnostic_tools.spec_sum)
# help(diagnostic_tools.spec_var)

## Variables

In [290]:
############### VARIABLES ###############

# Define dict for variable dimensions
spatial_dims = ('time','z','y','x')
spectral_dims = ('time','z','l','k')
dim_database = {
    'q': spatial_dims,
    'u': spatial_dims,
    'v': spatial_dims,
    'ufull': spatial_dims,
    'vfull': spatial_dims, 
    'qh': spectral_dims,
    'uh': spectral_dims,
    'vh': spectral_dims,
    'ph': spectral_dims, 
    'a': ('time','z','z','l','k'),
    'Ubg': ('time','z'),
    'Qy': ('time','z'),
    'kk': ('time','k'),
    'll': ('time','l'),
    
}

# dict for variable dimensions
var_attr_database = {
    'q': {'standard_name': 'q', 'description': 'potential vorticity in real space', 'units': '',},
    'u': {'standard_name': 'u', 'description': 'zonal velocity anomaly', 'units': '',},
    'v': {'standard_name': 'v', 'description': 'meridional velocity anomaly', 'units': '',},
    'ufull': {'standard_name': 'ufull', 'description': 'zonal full velocities in real space', 'units': '',},
    'vfull': {'standard_name': 'vfull', 'description': 'meridional full velocities in real space', 'units': '',},
    'qh': {'standard_name': 'qh', 'description': 'potential vorticity in spectral space', 'units': '',},
    'uh': {'standard_name': 'uh', 'description': 'zonal velocity anomaly in spectral space', 'units': '',},
    'vh': {'standard_name': 'vh', 'description': 'meridional velocity anomaly in spectral space', 'units': '',},
    'ph': {'standard_name': 'ph', 'description': 'streamfunction in spectral space', 'units': '',},
    'a': {'standard_name': 'a', 'description': 'inversion matrix', 'units': '',},
    'Ubg': {'standard_name': 'Ubg', 'description': 'background zonal velocity', 'units': '',},
    'Qy': {'standard_name': 'Qy', 'description': 'background potential vorticity gradient', 'units': '',} ,
    'kk': {'standard_name': 'kk', 'description': 'zonal wavenumbers', 'units': '',} ,
    'll': {'standard_name': 'll', 'description': 'meridional wavenumbers', 'units': '',} ,

}

############### COORDS ###############

# dict for coordinate dimensions
coord_database = {
    'time': np.array([m.t]),
    'z': np.arange(1,m.nz+1),
    'x': m.x[0,:],
    'y': m.y[:,0],
    'l': m.l[:,0],
    'k': m.k[0,:],

}

# dict for coordinate attributes 
coord_attr_database = {
    'time': {'standard_name': 'time', 'description': 'model time', 'units': '',},
    'z': {'standard_name': 'z', 'description': 'vertical levels', 'units': '',},
    'x': {'standard_name': 'x', 'description': '', 'units': '',},
    'y': {'standard_name': 'y', 'description': '', 'units': '',},
    'l': {'standard_name': 'l', 'description': '', 'units': '',},
    'k': {'standard_name': 'k', 'description': '', 'units': '',},

}

############### ATTRS ###############

# dict for dataset attributes
ds_attr_database = {
    'nx': {'standard_name': 'nx', 'description': 'number of real space grid points in x direction', 'units': '',},
    'ny': {'standard_name': 'ny', 'description': 'number of real space grid points in y direction (default: nx)', 'units': '',},
    'nz': {'standard_name': 'nz', 'description': 'number of vertical levels', 'units': '',},
    'nl': {'standard_name': 'nl', 'description': 'number of spectral space grid points in l direction', 'units': '',},
    'rek': {'standard_name': 'rek', 'description': 'linear drag in lower layer', 'units': 'seconds^(-1)',},
    'tc': {'standard_name': 'tc', 'description': 'model timestep', 'units': '',},
    'dt': {'standard_name': 'st', 'description': 'numerical timestep', 'units': '',},
    'L': {'standard_name': 'L', 'description': 'domain length in x direction', 'units': '',},
    'W': {'standard_name': 'W', 'description': 'domain length in y direction', 'units': '',},
    'filterfac': {'standard_name': 'filterfac', 'description': 'amplitude of spectral spherical filter', 'units': '',},
    'twrite': {'standard_name': 'twrite', 'description': 'interval for cfl writeout (units: number of timesteps)', 'units': '',},
    'tmax': {'standard_name': 'tmax', 'description': 'total time of integration', 'units': 'model time',},
    'tavestart': {'standard_name': 'tavestart', 'description': 'start time for averaging (units: model time)', 'units': '',},
    'tsnapstart': {'standard_name': 'tsnapstart', 'description': 'start time for snapshot writeout (units: model time)', 'units': ''},
    'taveint': {'standard_name': 'taveint', 'description': 'time interval for accumulation of diagnostic averages (units: model time)', 'units': ''},
    'tsnapint': {'standard_name': 'tsnapint', 'description': 'time interval for snapshots (units: model time)', 'units': '',},
    'ntd': {'standard_name': 'ntd', 'description': 'number of threads used', 'units': '',},
    'pmodes': {'standard_name': 'pmodes', 'description': 'vertical pressure modes (unitless)', 'units': '',},
    'radii': {'standard_name': 'radii', 'description': 'deformation radii (units: model length)', 'units': '',},
    
}


In [291]:
# Create list of variable, coordinate, and attribute dict entries
variables = {}
for vname in dim_database:
    data = getattr(m,vname)
    variables[vname] = (dim_database[vname],data[np.newaxis,...])
    
coordinates = {}
for cname in coord_database:
#     data = getattr(m,cname)
    coordinates[cname] = coord_database[cname] 
    
attributes = {}
for aname in ds_attr_database:
    try: 
        data = getattr(m,aname)
        attributes[aname] = (data, ds_attr_database[aname])
    except:
        pass

In [292]:
# Define dataset
ds = xr.Dataset(variables,
                coords=coordinates, 
               attrs=attributes)


In [293]:
# Assign attributes to coordinates
for caname in coord_attr_database:
    if caname in ds.coords:
        ds.coords[caname].attrs = coord_attr_database[caname]
        
# Assign attributes to variables
for vaname in var_attr_database:
    if vaname in ds.data_vars:
        ds.data_vars[vaname].attrs = var_attr_database[vaname]

In [297]:
ds

<xarray.Dataset>
Dimensions:  (k: 33, l: 64, time: 1, x: 64, y: 64, z: 2)
Coordinates:
  * time     (time) float64 3.11e+08
  * z        (z) int64 1 2
  * x        (x) float64 7.812e+03 2.344e+04 3.906e+04 ... 9.766e+05 9.922e+05
  * y        (y) float64 7.812e+03 2.344e+04 3.906e+04 ... 9.766e+05 9.922e+05
  * l        (l) float64 0.0 6.283e-06 1.257e-05 ... -1.257e-05 -6.283e-06
  * k        (k) float64 0.0 6.283e-06 1.257e-05 ... 0.0001948 0.0002011
Data variables: (12/14)
    q        (time, z, y, x) float64 -1.355e-05 -2.186e-05 ... 1.854e-06
    u        (time, z, y, x) float64 0.03474 0.01235 ... 0.009037 0.007816
    v        (time, z, y, x) float64 0.08101 0.04147 ... 0.002231 0.007576
    ufull    (time, z, y, x) float64 0.05974 0.03735 ... 0.009037 0.007816
    vfull    (time, z, y, x) float64 0.08101 0.04147 ... 0.002231 0.007576
    qh       (time, z, l, k) complex128 (0.0023229921630165905+0j) ... (2.320...
    ...       ...
    ph       (time, z, l, k) complex128 0j ... (4.18330719484164e-16+1.543053...
    a        (time, z, z, l, k) complex128 0j ... (-24223355.801269166+0j)
    Ubg      (time, z) float64 0.025 0.0
    Qy       (time, z) float64 1.039e-10 -7.222e-12
    kk       (time, k) float64 0.0 6.283e-06 1.257e-05 ... 0.0001948 0.0002011
    ll       (time, l) float64 0.0 6.283e-06 1.257e-05 ... -1.257e-05 -6.283e-06
Attributes: (12/15)
    nx:         (64, {'standard_name': 'nx', 'description': 'number of real s...
    ny:         (64, {'standard_name': 'ny', 'description': 'number of real s...
    nz:         (2, {'standard_name': 'nz', 'description': 'number of vertica...
    nl:         (64, {'standard_name': 'nl', 'description': 'number of spectr...
    rek:        (5.787e-07, {'standard_name': 'rek', 'description': 'linear d...
    tc:         (43200, {'standard_name': 'tc', 'description': 'model timeste...
    ...         ...
    filterfac:  (23.6, {'standard_name': 'filterfac', 'description': 'amplitu...
    twrite:     (10000, {'standard_name': 'twrite', 'description': 'interval ...
    tmax:       (311040000.0, {'standard_name': 'tmax', 'description': 'total...
    tavestart:  (155520000.0, {'standard_name': 'tavestart', 'description': '...
    taveint:    (86400.0, {'standard_name': 'taveint', 'description': 'time i...
    ntd:        (1, {'standard_name': 'ntd', 'description': 'number of thread...

In [298]:
len(ds.x)

64

In [299]:
m.nx

64